In [ ]:
import pandas as pd
import numpy as np
import datetime
import math
import os

# ==========================================
# CONFIGURATION
# ==========================================
FUT_FILE = "NIFTY20NOV.csv"                 # futures tick file
TRADES_FILE = "kinetic_full_backtest_trades.csv" # kinetic backtest output with Entry/Exit
OPTION_PREFIX = "NIFTY25NOV"                # prefix for options files
STRIKES = [26100, 26150, 26200, 26250, 26300]  # strikes you have
LOT_SIZE = 75                               # NIFTY lot
HEDGE_STEP_THRESHOLD = 0.02                 # min delta change to rebalance (in futures)
HEDGE_COST_PER_CONTRACT = 0.2               # points per futures contract per hedge (slippage+fees)
EXPIRY_DATE = pd.Timestamp("2025-11-27")    # <-- SET CORRECT EXPIRY HERE

print("\n==============================")
print("DELTA-HEDGED GAMMA SCALPING")
print("==============================\n")

# ==========================================
# HELPER: NORMAL PDF & CDF
# ==========================================
def norm_pdf(x):
    return 1.0 / math.sqrt(2 * math.pi) * math.exp(-0.5 * x * x)

def norm_cdf(x):
    # Using error function approximation
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))


# ==========================================
# BLACK-SCHOLES: CALL PRICE & DELTA
# ==========================================
def bs_call_price(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(S - K, 0.0)
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

def bs_call_delta(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return 1.0 if S > K else 0.0
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    return norm_cdf(d1)


# ==========================================
# IMPLIED VOL (BISECTION)
# ==========================================
def implied_vol_call_bisect(market_price, S, K, T, r=0.0, 
                            sigma_low=0.0001, sigma_high=3.0, 
                            tol=1e-4, max_iter=50):
    """Simple bisection implied vol for call. Returns sigma or None."""
    if T <= 0:
        return None

    # Check bounds
    price_low = bs_call_price(S, K, T, sigma_low, r)
    price_high = bs_call_price(S, K, T, sigma_high, r)
    if price_low > market_price or price_high < market_price:
        # Outside theoretical range, fallback
        return None

    for _ in range(max_iter):
        sigma_mid = 0.5 * (sigma_low + sigma_high)
        price_mid = bs_call_price(S, K, T, sigma_mid, r)

        if abs(price_mid - market_price) < tol:
            return sigma_mid

        if price_mid > market_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid

    return sigma_mid


# ==========================================
# LOAD FUTURES DATA (NIFTY20NOV.csv)
# ==========================================
print("Loading futures data...")
fut_df = pd.read_csv(FUT_FILE)

# Expecting columns: 'Date', 'Time', 'LTP'
# Adjust if your columns have different names
fut_df['DateTime'] = pd.to_datetime(
    fut_df['Date'] + ' ' + fut_df['Time'],
    format='%d/%m/%Y %H:%M:%S.%f',
    errors='coerce'
)
fut_df = fut_df.dropna(subset=['DateTime'])
fut_df['LTP'] = pd.to_numeric(fut_df['LTP'], errors='coerce')
fut_df = fut_df.dropna(subset=['LTP'])
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)

print(f"Futures ticks: {len(fut_df):,}")
print(f"From {fut_df['DateTime'].min()} to {fut_df['DateTime'].max()}\n")


# ==========================================
# LOAD KINETIC TRADES (ENTRY/EXIT WINDOWS)
# ==========================================
print("Loading kinetic trade windows...")
trades_raw = pd.read_csv(TRADES_FILE)

# Expecting 'Entry_Time' and 'Exit_Time'
trades_raw['Entry_Time'] = pd.to_datetime(trades_raw['Entry_Time'])
trades_raw['Exit_Time']  = pd.to_datetime(trades_raw['Exit_Time'])

# Filter only trades for this specific day (20 Nov) if needed:
# trades_raw = trades_raw[trades_raw['Entry_Time'].dt.date == datetime.date(2025, 11, 20)]

trades_raw = trades_raw.sort_values('Entry_Time').reset_index(drop=True)
print(f"Number of kinetic trades in file: {len(trades_raw)}\n")


# ==========================================
# LOAD OPTION FILES INTO A SINGLE DATAFRAME
# ==========================================
print("Loading option data (CE + PE for strikes)...")
opt_frames = []

for K in STRIKES:
    for opt_type in ['CE', 'PE']:
        fname = f"{OPTION_PREFIX}{K}{opt_type}.parquet"
        if not os.path.exists(fname):
            print(f"WARNING: Missing file {fname}, skipping.")
            continue
        
        df_opt = pd.read_parquet(fname)
        # Expecting 'Date', 'Time', 'LTP' similarly
        df_opt['DateTime'] = pd.to_datetime(
            df_opt['Date'] + ' ' + df_opt['Time'],
            format='%d/%m/%Y %H:%M:%S.%f',
            errors='coerce'
        )
        df_opt = df_opt.dropna(subset=['DateTime'])
        df_opt['LTP'] = pd.to_numeric(df_opt['LTP'], errors='coerce')
        df_opt = df_opt.dropna(subset=['LTP'])

        df_opt['Strike'] = float(K)
        df_opt['Option_Type'] = opt_type  # 'CE' or 'PE'
        opt_frames.append(df_opt[['DateTime', 'LTP', 'Strike', 'Option_Type']])

opt_df = pd.concat(opt_frames, ignore_index=True).sort_values('DateTime')
print(f"Total option ticks: {len(opt_df):,}\n")


# ==========================================
# HELPER: GET SPOT AT TIME (ASOF)
# ==========================================
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)
fut_df.set_index('DateTime', inplace=True)

opt_df = opt_df.sort_values('DateTime').reset_index(drop=True)
opt_df.set_index('DateTime', inplace=True)


def get_spot_at(ts):
    """Get futures LTP at or just before ts."""
    if ts < fut_df.index[0]:
        return None
    loc = fut_df.index.searchsorted(ts, side='right') - 1
    if loc < 0:
        return None
    return fut_df.iloc[loc]['LTP']


def slice_ts(df, start_ts, end_ts):
    """Slice df between start_ts and end_ts inclusive."""
    return df.loc[(df.index >= start_ts) & (df.index <= end_ts)].copy()


# ==========================================
# GAMMA SCALPING SIM FOR ONE TRADE WINDOW
# ==========================================
def gamma_scalp_for_trade(row):
    entry_ts = row['Entry_Time']
    exit_ts  = row['Exit_Time']

    # 1) Get spot at entry
    S0 = get_spot_at(entry_ts)
    if S0 is None:
        return None

    # 2) Pick ATM strike from list (closest to S0)
    atm_strike = min(STRIKES, key=lambda K: abs(K - S0))

    # 3) Extract CE & PE series for that strike in [entry, exit]
    ce_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'CE')],
        entry_ts, exit_ts
    )
    pe_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'PE')],
        entry_ts, exit_ts
    )

    # Require some data
    if ce_series.empty or pe_series.empty:
        return None

    # 4) Build 1-second grid
    time_index = pd.date_range(start=entry_ts, end=exit_ts, freq='1S')

    # Reindex futures & options to this grid
    fut_sub = slice_ts(fut_df, entry_ts, exit_ts)
    if fut_sub.empty:
        return None

    S_t = fut_sub['LTP'].reindex(time_index, method='ffill')
    CE_t = ce_series['LTP'].reindex(time_index, method='ffill')
    PE_t = pe_series['LTP'].reindex(time_index, method='ffill')

    # Drop any times where we don't have all
    valid = S_t.notna() & CE_t.notna() & PE_t.notna()
    S_t = S_t[valid]
    CE_t = CE_t[valid]
    PE_t = PE_t[valid]
    time_index = S_t.index

    if len(time_index) < 5:
        return None

    # 5) ENTRY
    S_entry = S_t.iloc[0]
    CE_entry = CE_t.iloc[0]
    PE_entry = PE_t.iloc[0]

    # Time to expiry at entry
    T0 = (EXPIRY_DATE - time_index[0]).total_seconds() / (365.0 * 24 * 3600)
    T0 = max(T0, 1e-6)

    K = atm_strike

    # Implied vol for CE at entry
    sigma_ce = implied_vol_call_bisect(CE_entry, S_entry, K, T0)
    # For PE, we can use put-call parity to infer call-equivalent price:
    # C = P + S - K e^{-rT}, so treat that as "call price" for iv
    ce_equiv_from_put = PE_entry + S_entry - K  # r≈0
    sigma_pe = implied_vol_call_bisect(max(ce_equiv_from_put, 0.0001), S_entry, K, T0)

    if sigma_ce is None or sigma_pe is None:
        # fallback: use some rough vol (e.g. 0.15)
        sigma_ce = sigma_ce or 0.15
        sigma_pe = sigma_pe or 0.15

    # We are long 1 CE and 1 PE
    option_cost = CE_entry + PE_entry

    # 6) Start delta hedge
    hedge_pos = 0.0      # futures contracts
    hedge_cash_pnl = 0.0 # cash from futures trades
    hedge_trades = 0

    # 7) Time loop
    for t in time_index:
        S = S_t.loc[t]
        CE = CE_t.loc[t]
        PE = PE_t.loc[t]

        # Time to expiry at time t
        T_t = (EXPIRY_DATE - t).total_seconds() / (365.0 * 24 * 3600)
        T_t = max(T_t, 1e-6)

        # Call delta using CE's implied vol (gamma mostly from CE for ATM)
        delta_call = bs_call_delta(S, K, T_t, sigma_ce)
        # Put delta from call delta via parity: Δ_put = Δ_call - 1
        delta_put = delta_call - 1.0

        opt_delta = delta_call + delta_put  # total delta of straddle

        # Desired futures position to be net delta ~ 0
        desired_hedge_pos = -opt_delta  # 1 future = 1 delta per point

        # Adjust only if significant change
        delta_change = desired_hedge_pos - hedge_pos
        if abs(delta_change) > HEDGE_STEP_THRESHOLD:
            # Trade this much futures at price S
            # Negative delta_change => we buy futures (spend cash)
            hedge_cash_pnl -= delta_change * S
            hedge_pos = desired_hedge_pos
            hedge_trades += abs(delta_change)

    # 8) At EXIT: mark to market
    S_exit = S_t.iloc[-1]
    CE_exit = CE_t.iloc[-1]
    PE_exit = PE_t.iloc[-1]

    option_pnl = (CE_exit + PE_exit) - option_cost
    futures_pnl = hedge_pos * S_exit + hedge_cash_pnl

    gross_pnl_points = option_pnl + futures_pnl

    # Hedge costs: per-contract per trade in points
    # Treat hedge_trades as "futures contracts traded" approx
    hedge_cost_points = hedge_trades * HEDGE_COST_PER_CONTRACT

    net_pnl_points = gross_pnl_points - hedge_cost_points
    net_pnl_rupees = net_pnl_points * LOT_SIZE

    return {
        "Date": entry_ts.date(),
        "Entry_Time": entry_ts,
        "Exit_Time": exit_ts,
        "ATM_Strike": atm_strike,
        "S_Entry": S_entry,
        "S_Exit": S_exit,
        "CE_Entry": CE_entry,
        "CE_Exit": CE_exit,
        "PE_Entry": PE_entry,
        "PE_Exit": PE_exit,
        "Sigma_CE": sigma_ce,
        "Sigma_PE": sigma_pe,
        "Option_PnL_pts": option_pnl,
        "Futures_PnL_pts": futures_pnl,
        "Gross_PnL_pts": gross_pnl_points,
        "Hedge_Trades": hedge_trades,
        "Hedge_Cost_pts": hedge_cost_points,
        "Net_PnL_pts": net_pnl_points,
        "Net_PnL_rupees": net_pnl_rupees
    }


# ==========================================
# RUN OVER ALL TRADES
# ==========================================
results = []
for i, row in trades_raw.iterrows():
    res = gamma_scalp_for_trade(row)
    if res is not None:
        results.append(res)
    else:
        print(f"Skipped trade {i} (insufficient data or mismatch).")

if not results:
    print("\nNo valid gamma-scalping simulations completed.")
else:
    res_df = pd.DataFrame(results)
    print("\n========= GAMMA SCALPING SUMMARY =========")
    print(res_df[['Date', 'Entry_Time', 'Exit_Time', 'ATM_Strike',
                  'Net_PnL_pts', 'Net_PnL_rupees']].head())

    total_pts = res_df['Net_PnL_pts'].sum()
    total_rs  = res_df['Net_PnL_rupees'].sum()
    print("\nTOTAL Net Gamma PnL:")
    print(f"  {total_pts:.2f} points")
    print(f"  ₹{total_rs:,.2f}")

    print("\nPer-trade stats:")
    print(f"  Trades: {len(res_df)}")
    print(f"  Avg per trade: {res_df['Net_PnL_pts'].mean():.2f} pts "
          f"(₹{res_df['Net_PnL_rupees'].mean():.2f})")
    print(f"  Win rate: "
          f"{(res_df['Net_PnL_pts'] > 0).mean() * 100:.1f}%")

    # Save to CSV
    out_file = "gamma_scalping_results.csv"
    res_df.to_csv(out_file, index=False)
    print(f"\nDetailed gamma scalping results saved to: {out_file}")
